<a href="https://colab.research.google.com/github/OzcanCevik/Colab/blob/main/roboflow_vericekme_modele%C4%9Fitme_otomatik.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install roboflow

In [2]:
from roboflow import Roboflow

In [3]:
rf = Roboflow(api_key="fKsVVo3aUM2HEy2TkeFA")

In [4]:
project = rf.workspace("wordalpha02").project("wordalpha2-gkrmd")

loading Roboflow workspace...
loading Roboflow project...


In [5]:
version = project.version(3)

In [6]:
dataset = version.download("yolov8")

[WARNING] we noticed you are downloading a `yolov8` datasets but you don't have `ultralytics` installed. Roboflow `.deploy` supports only models trained with `ultralytics==8.0.196`, to intall it `pip install ultralytics==8.0.196`.



Extracting Dataset Version Zip to wordalpha2-3 in yolov8:: 100%|██████████| 29744/29744 [00:04<00:00, 6077.54it/s]


In [ ]:
!pip install ultralytics

In [8]:
from ultralytics import YOLO

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
model = YOLO("yolov8n.pt")

100%|██████████| 6.25M/6.25M [00:00<00:00, 23.6MB/s]


In [ ]:
import zipfile
import os
# İndirilen ZIP dosyasının yolunu belirtin
zip_path = "/content/wordalpha2-1/roboflow.zip"
output_dir = "/content/dataset"


In [ ]:
import shutil

# Klasörü sil runs
shutil.rmtree('/content/runs')


In [ ]:
model.train(data='/content/wordalpha2-3/data.yaml', epochs=50, imgsz=640)

In [ ]:
from ultralytics import YOLO  # Gerekli kütüphaneyi içe aktar

# Modelinizi yükleyin
model = YOLO('/content/yolov8n.pt')  # Model dosyanızın yolu

# Eğitim devam ettirme
try:
    model.train(data='/content/wordalpha2-3/data.yaml',
                epochs=50,  # Toplam dönem sayısı
                imgsz=640,
                resume='/content/runs/detect/train/weights/last.pt')  # Kaldığınız yerden devam edin
except AssertionError as e:
    print(f"Hata: {e}")  # Hata mesajını yazdır


In [ ]:
import os

# Model dosyasının varlığını kontrol et
if os.path.exists('/content/runs/detect/train/weights/best.pt'):
    print("Model dosyası mevcut.")
else:
    print("Model dosyası mevcut değil. Lütfen yolu kontrol edin.")


Model dosyası mevcut.


In [9]:
import os
import shutil
import random
from glob import glob
from pathlib import Path
import yaml

# Orijinal veri seti yolunu belirleyin
data_dir = "/content/wordalpha2-3"
train_images = sorted(glob(os.path.join(data_dir, "train/images/*.jpg")))
train_labels = sorted(glob(os.path.join(data_dir, "train/labels/*.txt")))
val_images = sorted(glob(os.path.join(data_dir, "val/images/*.jpg")))
val_labels = sorted(glob(os.path.join(data_dir, "val/labels/*.txt")))
test_images = sorted(glob(os.path.join(data_dir, "test/images/*.jpg")))
test_labels = sorted(glob(os.path.join(data_dir, "test/labels/*.txt")))

# Veri setini bölme fonksiyonu
def split_data(images, labels, n_splits=4):
    data = list(zip(images, labels))
    random.shuffle(data)
    split_size = len(data) // n_splits
    return [data[i * split_size:(i + 1) * split_size] for i in range(n_splits)]

# Veriyi dört parçaya bölme
train_splits = split_data(train_images, train_labels)
val_splits = split_data(val_images, val_labels)
test_splits = split_data(test_images, test_labels)

# Yeni klasörler oluşturma ve veriyi kaydetme
output_dir = "/content/SplitData"
os.makedirs(output_dir, exist_ok=True)

for i in range(4):
    split_dir = os.path.join(output_dir, f"split_{i+1}")
    os.makedirs(split_dir, exist_ok=True)
    os.makedirs(os.path.join(split_dir, "train/images"), exist_ok=True)
    os.makedirs(os.path.join(split_dir, "train/labels"), exist_ok=True)
    os.makedirs(os.path.join(split_dir, "val/images"), exist_ok=True)
    os.makedirs(os.path.join(split_dir, "val/labels"), exist_ok=True)
    os.makedirs(os.path.join(split_dir, "test/images"), exist_ok=True)
    os.makedirs(os.path.join(split_dir, "test/labels"), exist_ok=True)

    for img, lbl in train_splits[i]:
        shutil.copy(img, os.path.join(split_dir, "train/images", os.path.basename(img)))
        shutil.copy(lbl, os.path.join(split_dir, "train/labels", os.path.basename(lbl)))

    for img, lbl in val_splits[i]:
        shutil.copy(img, os.path.join(split_dir, "val/images", os.path.basename(img)))
        shutil.copy(lbl, os.path.join(split_dir, "val/labels", os.path.basename(lbl)))

    for img, lbl in test_splits[i]:
        shutil.copy(img, os.path.join(split_dir, "test/images", os.path.basename(img)))
        shutil.copy(lbl, os.path.join(split_dir, "test/labels", os.path.basename(lbl)))

    # data.yaml dosyasını kopyalama ve düzenleme
    with open(os.path.join(data_dir, "data.yaml"), 'r') as file:
        data_yaml = yaml.safe_load(file)
    data_yaml['train'] = os.path.join(split_dir, "train/images")
    data_yaml['val'] = os.path.join(split_dir, "val/images")
    data_yaml['test'] = os.path.join(split_dir, "test/images")

    with open(os.path.join(split_dir, "data.yaml"), 'w') as file:
        yaml.dump(data_yaml, file)


# Klasörleri zipleyerek bilgisayara indirme
shutil.make_archive("/content/SplitData", 'zip', output_dir)

'/content/SplitData.zip'

In [10]:
import shutil

# Klasörü zip dosyasına dönüştürme
shutil.make_archive("/content/SplitData", 'zip', "/content/SplitData")


'/content/SplitData.zip'

In [11]:
from google.colab import files

# Zip dosyasını indir
files.download("/content/SplitData.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [13]:
from ultralytics import YOLO
import torch
import shutil

# CUDA'nın mevcut olup olmadığını kontrol edin
use_cuda = torch.cuda.is_available()
device = 'cuda' if use_cuda else 'cpu'
print(f"Eğitim için kullanılan cihaz: {device}")

# Eğitimin yapılacağı veri seti yolunu belirleyin
split_data_dir = "/content/SplitData"

# Eğitim döngüsü
model_path = "yolov8n.pt"
for i in range(1, 5):
    # Modeli yükleyin veya önceki eğitimden devam edin
    if i == 1:
        model = YOLO(model_path)
    else:
        model = YOLO(f"yolov8n_epoch_{(i-1)*25}.pt")

    # Eğitimi başlatın
    data_path = os.path.join(split_data_dir, f"split_{i}", "data.yaml")
    results = model.train(
        data=data_path,
        epochs=25,
        imgsz=640,
        batch=16,
        device=device
    )

    # Modeli ve eğitim durumunu kaydedin
    model_path = f"yolov8n_epoch_{i*25}.pt"
    model.save(model_path)

    # Model dosyasını indir
    shutil.move(model_path, f"/content/{model_path}")

    from google.colab import files
    files.download(f"/content/{model_path}")


Eğitim için kullanılan cihaz: cuda
Ultralytics YOLOv8.2.103 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/SplitData/split_1/data.yaml, epochs=25, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=cuda, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=

100%|██████████| 755k/755k [00:00<00:00, 25.2MB/s]


Overriding model.yaml nc=80 with nc=62

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytic

train: Scanning /content/SplitData/split_1/train/labels... 3563 images, 4 backgrounds, 5 corrupt: 100%|██████████| 3568/3568 [00:02<00:00, 1619.09it/s]

train: WARNING ⚠️ /content/SplitData/split_1/train/images/1389-00_jpg.rf.47ba28364a39399a11fe990468860a3e.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /content/SplitData/split_1/train/images/260-15_jpg.rf.200bcfba123b430d9d3b23527ec79989.jpg: ignoring corrupt image/label: image size (15, 7) <10 pixels
train: WARNING ⚠️ /content/SplitData/split_1/train/images/260-15_jpg.rf.6617b1368c524853161e1abcb4316253.jpg: ignoring corrupt image/label: image size (15, 7) <10 pixels
train: WARNING ⚠️ /content/SplitData/split_1/train/images/327-32_jpg.rf.1aa5108a92c1bb4c76c1f278752e8006.jpg: ignoring corrupt image/label: image size (15, 7) <10 pixels
train: WARNING ⚠️ /content/SplitData/split_1/train/images/327-32_jpg.rf.8389cf9bb4a61b4abeb22e170d350c2c.jpg: ignoring corrupt image/label: image size (15, 7) <10 pixels
train: WARNING ⚠️ /content/SplitData/split_1/train/images/610-33_jpg.rf.dd8c1fc2400e244fade54f202e24e86b.jpg: ignoring corrupt image/label: image size (13, 9) <10 pixels


...

                   all        135        756      0.951      0.963      0.971      0.859

25 epochs completed in 0.630 hours.
Optimizer stripped from runs/detect/train4/weights/last.pt, 6.3MB
Optimizer stripped from runs/detect/train4/weights/best.pt, 6.3MB

Validating runs/detect/train4/weights/best.pt...
Ultralytics YOLOv8.2.103 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3,017,738 parameters, 0 gradients, 8.2 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:05<00:00,  1.02s/it]


                   all        135        756      0.958      0.959      0.974      0.863
                     0          5          7      0.973      0.857       0.93      0.849
                     1         12         13      0.923          1      0.984       0.86
                     2         14         18       0.99          1      0.995        0.9
                     3         17         20       0.99          1      0.995      0.913
                     4          4          4      0.954          1      0.995      0.952
                     5          9         13      0.987          1      0.995      0.897
                     6          9         12       0.99          1      0.995       0.94
                     7          9         14       0.99          1      0.995      0.888
                     8          7         11      0.981          1      0.995       0.93
                     9          3          4      0.996          1      0.995      0.859
                   A_

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>